**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

**Exploración y Curación de Datos**

*Edición 2022*

----

# Trabajo práctico entregable - parte 1

En esta notebook, vamos a cargar el conjunto de datos de [la compentencia Kaggle](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot) sobre estimación de precios de ventas de propiedades en Melbourne, Australia.

Utilizaremos el conjunto de datos reducido producido por [DanB](https://www.kaggle.com/dansbecker). Hemos subido una copia a un servidor de la Universidad Nacional de Córdoba para facilitar su acceso remoto.

In [1]:
import matplotlib.pyplot as plt
import numpy
import pandas as pd

import seaborn
seaborn.set_context('talk')

from decouple import config
from sqlalchemy import create_engine, text

In [2]:
import plotly
plotly.__version__
# Make sure it's 4.14.3

'5.8.1'

In [3]:
melb_df = pd.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv')
melb_df[:3]

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0


## Ejercicio 1 SQL: 

1. Crear una base de datos en SQLite utilizando la libreria [SQLalchemy](https://stackoverflow.com/questions/2268050/execute-sql-from-file-in-sqlalchemy).
https://docs.sqlalchemy.org/en/14/core/engines.html#sqlite

2. Ingestar los datos provistos en 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv' en una tabla y el dataset generado en clase con datos de airbnb y sus precios por codigo postal en otra.

3. Implementar consultas en SQL que respondan con la siguiente información:

    - cantidad de registros totales por ciudad.
    - cantidad de registros totales por barrio y ciudad.

4. Combinar los datasets de ambas tablas ingestadas utilizando el comando JOIN de SQL  para obtener un resultado similar a lo realizado con Pandas en clase.  



In [4]:
# Ejercicio 1.1: creamos una SQLite DB usando SQL Alchemy
engine = create_engine('sqlite:///db.sqlite3', echo=True)

In [5]:
# Abrimos el dataset de Airbnb de precios por código postal
interesting_cols = [
  'description', 'neighborhood_overview',
  'street', 'neighborhood', 'city', 'suburb', 'state', 'zipcode',
  'price', 'weekly_price', 'monthly_price',
  'latitude', 'longitude',
]

airbnb_df = pd.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv',
    usecols=interesting_cols)

airbnb_df.head()

/run/user/1000/app/com.jetbrains.PyCharm-Community/ipykernel_668/2536014331.py:9: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  airbnb_df = pd.read_csv(


,description,neighborhood_overview,street,neighborhood,city,suburb,state,zipcode,latitude,longitude,price,weekly_price,monthly_price
0,"House: Clean, New, Modern, Quite, Safe. 10Km f...",Very safe! Family oriented. Older age group.,"Bulleen, VIC, Australia",Balwyn North,Manningham,Bulleen,VIC,3105,-37.772684,145.092133,60,NaN,NaN
1,A large air conditioned room with queen spring...,This hip area is a crossroads between two grea...,"Brunswick East, VIC, Australia",Brunswick,Moreland,Brunswick East,VIC,3057,-37.766505,144.980736,35,200.0,803.0
2,RIGHT IN THE HEART OF ST KILDA! It doesn't get...,A stay at our apartment means you can enjoy so...,"St Kilda, VIC, Australia",St Kilda,Port Phillip,St Kilda,VIC,3182,-37.859755,144.977369,159,1253.0,4452.0
3,"Comfortable, relaxed house, a home away from ...","This is a great neighbourhood – it is quiet, y...","Thornbury, VIC, Australia",Thornbury,Darebin,Thornbury,VIC,3071,-37.758971,144.989228,50,250.0,920.0
4,We offer comfortable accommodation in Inner Me...,NaN,"St Kilda East, VIC, Australia",St Kilda East,Port Phillip,St Kilda East,VIC,3183,-37.864530,144.992238,69,400.0,1900.0


In [6]:
# Ingestamos los datasets en tablas

melb_df.to_sql('property_sales_prices', con=engine, if_exists="replace")

airbnb_df.to_sql('airbnb_property_prices', con=engine, if_exists="replace")

2022-06-08 18:09:24,518 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("property_sales_prices")
2022-06-08 18:09:24,518 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-08 18:09:24,519 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("property_sales_prices")
2022-06-08 18:09:24,520 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-08 18:09:24,522 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-08 18:09:24,524 INFO sqlalchemy.engine.Engine 
CREATE TABLE property_sales_prices (
	"index" BIGINT, 
	"Suburb" TEXT, 
	"Address" TEXT, 
	"Rooms" BIGINT, 
	"Type" TEXT, 
	"Price" FLOAT, 
	"Method" TEXT, 
	"SellerG" TEXT, 
	"Date" TEXT, 
	"Distance" FLOAT, 
	"Postcode" FLOAT, 
	"Bedroom2" FLOAT, 
	"Bathroom" FLOAT, 
	"Car" FLOAT, 
	"Landsize" FLOAT, 
	"BuildingArea" FLOAT, 
	"YearBuilt" FLOAT, 
	"CouncilArea" TEXT, 
	"Lattitude" FLOAT, 
	"Longtitude" FLOAT, 
	"Regionname" TEXT, 
	"Propertycount" FLOAT
)


2022-06-08 18:09:24,525 INFO sqlalchemy.engine.Engine [no key 0.00053s] (

2022-06-08 18:09:25,045 INFO sqlalchemy.engine.Engine COMMIT


22895

In [7]:
#Implementar consultas en SQL que respondan con la siguiente información:
#cantidad de registros totales por ciudad.
#cantidad de registros totales por barrio y ciudad.

# Armamos las consultas

query_melb_city = """
SELECT 'Melbourne' AS 'city', NULL AS 'suburb', COUNT(*) AS 'count'
FROM property_sales_prices
ORDER BY count DESC
"""
query_melb_suburbs = """
SELECT 'Melbourne' AS 'city', suburb AS 'suburb', COUNT(*) AS 'count'
FROM property_sales_prices
GROUP BY suburb
ORDER BY count DESC
"""
query_airbnb_city = """
SELECT city AS 'city', NULL AS 'suburb', COUNT(*) AS 'count'
FROM airbnb_property_prices
GROUP BY city
ORDER BY count DESC
"""

query_airbnb_suburbs = """
SELECT city AS 'city', suburb AS 'suburb', COUNT(*) AS 'count'
FROM airbnb_property_prices
GROUP BY city, suburb
ORDER BY count DESC
"""

queries = [
    {'description': 'Registros totales por ciudad (Melbourne data)',
     'query': query_melb_city}, 
    {'description': 'Registros totales por ciudad y barrio (Melbourne data)',
     'query':query_melb_suburbs},
    {'description': 'Registros totales por ciudad (Airbnb data)',
     'query':query_airbnb_city},
    {'description': 'Registros totales por ciudad y barrio (Airbnb data)',
     'query':query_airbnb_suburbs},
]

In [8]:
# Ejecutamos las consultas
with engine.connect() as connection:
    for item in queries:
        result = connection.execute(item['query'])
        print(f"\n{item['description']}\n")
        for row in result:
            suburb = "" if row['suburb'] is None else f" - {row['suburb']}"
            print(f"{row['city']}{suburb}: {row['count']}")
        print("\n")
        

2022-06-08 18:09:25,071 INFO sqlalchemy.engine.Engine 
SELECT 'Melbourne' AS 'city', NULL AS 'suburb', COUNT(*) AS 'count'
FROM property_sales_prices
ORDER BY count DESC

2022-06-08 18:09:25,071 INFO sqlalchemy.engine.Engine [raw sql] ()

Registros totales por ciudad (Melbourne data)

Melbourne: 13580


2022-06-08 18:09:25,073 INFO sqlalchemy.engine.Engine 
SELECT 'Melbourne' AS 'city', suburb AS 'suburb', COUNT(*) AS 'count'
FROM property_sales_prices
GROUP BY suburb
ORDER BY count DESC

2022-06-08 18:09:25,073 INFO sqlalchemy.engine.Engine [raw sql] ()

Registros totales por ciudad y barrio (Melbourne data)

Melbourne - Reservoir: 359
Melbourne - Richmond: 260
Melbourne - Bentleigh East: 249
Melbourne - Preston: 239
Melbourne - Brunswick: 222
Melbourne - Essendon: 220
Melbourne - South Yarra: 202
Melbourne - Glen Iris: 195
Melbourne - Hawthorn: 191
Melbourne - Coburg: 190
Melbourne - Northcote: 188
Melbourne - Brighton: 186
Melbourne - Kew: 177
Melbourne - Pascoe Vale: 171
Melbourne 

2022-06-08 18:09:25,078 INFO sqlalchemy.engine.Engine [raw sql] ()

Registros totales por ciudad (Airbnb data)

Melbourne: 7368
Port Phillip: 2808
Yarra: 2049
Stonnington: 1621
Moreland: 967
Yarra Ranges: 771
Darebin: 698
Boroondara: 664
Glen Eira: 631
Whitehorse: 614
Monash: 571
Maribyrnong: 436
Wyndham: 426
Bayside: 375
Moonee Valley: 344
Manningham: 313
Kingston: 309
Hobsons Bay: 239
Banyule: 203
Frankston: 177
Knox: 175
Hume: 170
Casey: 153
Greater Dandenong: 147
Whittlesea: 137
Cardinia: 123
Maroondah: 115
Brimbank: 108
Melton: 95
Nillumbik: 88


2022-06-08 18:09:25,089 INFO sqlalchemy.engine.Engine 
SELECT city AS 'city', suburb AS 'suburb', COUNT(*) AS 'count'
FROM airbnb_property_prices
GROUP BY city, suburb
ORDER BY count DESC

2022-06-08 18:09:25,089 INFO sqlalchemy.engine.Engine [raw sql] ()

Registros totales por ciudad y barrio (Airbnb data)

Melbourne - Melbourne: 3850
Melbourne - Southbank: 1190
Port Phillip - Saint Kilda: 802
Stonnington - South Yarra: 790
Melbourne - D

In [27]:
#Combinar los datasets de ambas tablas ingestadas utilizando el comando JOIN de SQL
#para obtener un resultado similar a lo realizado con Pandas en clase.

query = """
SELECT  P.Suburb, P.Address, P.Rooms, P.Type, P.Method, P.SellerG, P.Date,
        P.Distance, P.Postcode, P.Bedroom2, P.Bathroom, P.Car, P.Landsize,
        P.BuildingArea, P.YearBuilt, P.CouncilArea, P.Lattitude, P.Longtitude,
        P.Regionname, P.Propertycount,
        A.zipcode, A.airbnb_price_mean, A.airbnb_record_count, A.airbnb_weekly_price_mean, A.airbnb_monthly_price_mean

FROM property_sales_prices AS P 

LEFT OUTER JOIN 
(
    SELECT  CAST(zipcode AS INT) AS zipcode,
            AVG(price) AS airbnb_price_mean, 
            COUNT(zipcode) AS airbnb_record_count,
            AVG(weekly_price) AS airbnb_weekly_price_mean, 
            AVG(monthly_price) AS airbnb_monthly_price_mean
    FROM airbnb_property_prices
    GROUP BY CAST(zipcode AS INT)
    HAVING zipcode NOT NULL
) AS A ON CAST(P.postcode AS INT) = A.zipcode

"""



with engine.connect() as connection:
    result = connection.execute(query)
    for row in result:
        print(row)

2022-06-08 19:00:27,408 INFO sqlalchemy.engine.Engine 
SELECT  P.Suburb, P.Address, P.Rooms, P.Type, P.Method, P.SellerG, P.Date,
        P.Distance, P.Postcode, P.Bedroom2, P.Bathroom, P.Car, P.Landsize,
        P.BuildingArea, P.YearBuilt, P.CouncilArea, P.Lattitude, P.Longtitude,
        P.Regionname, P.Propertycount,
        A.zipcode, A.airbnb_price_mean, A.airbnb_record_count, A.airbnb_weekly_price_mean, A.airbnb_monthly_price_mean

FROM property_sales_prices AS P 

LEFT OUTER JOIN 
(
    SELECT  CAST(zipcode AS INT) AS zipcode,
            AVG(price) AS airbnb_price_mean, 
            COUNT(zipcode) AS airbnb_record_count,
            AVG(weekly_price) AS airbnb_weekly_price_mean, 
            AVG(monthly_price) AS airbnb_monthly_price_mean
    FROM airbnb_property_prices
    GROUP BY CAST(zipcode AS INT)
    HAVING zipcode NOT NULL
) AS A ON CAST(P.postcode AS INT) = A.zipcode


2022-06-08 19:00:27,409 INFO sqlalchemy.engine.Engine [raw sql] ()
('Abbotsford', '85 Turner St', 2,

('Werribee', '12 Bridge St', 2, 'h', 'S', 'YPA', '29/07/2017', 14.7, 3030.0, 2.0, 1.0, 1.0, 487.0, None, None, 'Wyndham', -37.90379, 144.66165, 'Western Metropolitan', 16166.0, 3030, 115.74806201550388, 258, 590.0, 2224.6)
('Werribee', '18 Conquest Dr', 4, 'h', 'PI', 'hockingstuart', '29/07/2017', 14.7, 3030.0, 4.0, 2.0, 2.0, 555.0, 237.0, 2002.0, 'Wyndham', -37.89359, 144.66911000000005, 'Western Metropolitan', 16166.0, 3030, 115.74806201550388, 258, 590.0, 2224.6)
('Werribee', '46 Coventry Dr', 4, 'h', 'S', 'YPA', '29/07/2017', 14.7, 3030.0, 4.0, 2.0, 0.0, 533.0, 170.0, 1980.0, 'Wyndham', -37.89337, 144.64003, 'Western Metropolitan', 16166.0, 3030, 115.74806201550388, 258, 590.0, 2224.6)
('Werribee', '13b Kingfisher Ct', 3, 'u', 'PI', 'hockingstuart', '29/07/2017', 14.7, 3030.0, 3.0, 2.0, 4.0, 287.0, 137.0, 2013.0, 'Wyndham', -37.87895, 144.6707, 'Western Metropolitan', 16166.0, 3030, 115.74806201550388, 258, 590.0, 2224.6)
('Werribee', '40 Tarneit Rd', 3, 'h', 'S', 'Ray', '29/07/201

('Essendon', '3/1 Cameron Rd', 4, 't', 'S', 'Barry', '3/09/2017', 7.5, 3040.0, 4.0, 3.0, 1.0, 205.0, 168.0, 2010.0, None, -37.74588, 144.92700000000005, 'Western Metropolitan', 9264.0, 3040, 114.14285714285714, 56, 787.5, 3366.6666666666665)
('Essendon', '24 Fletcher St', 4, 'h', 'VB', 'Nelson', '3/09/2017', 7.5, 3040.0, 4.0, 2.0, 3.0, 920.0, 256.0, 1990.0, None, -37.7554, 144.92351000000005, 'Western Metropolitan', 9264.0, 3040, 114.14285714285714, 56, 787.5, 3366.6666666666665)
('Essendon', '104 Hoffmans Rd', 4, 'h', 'S', 'Barry', '3/09/2017', 7.5, 3040.0, 4.0, 1.0, 1.0, 478.0, None, None, None, -37.74424000000001, 144.89149, 'Western Metropolitan', 9264.0, 3040, 114.14285714285714, 56, 787.5, 3366.6666666666665)
('Essendon', '126 Market St', 3, 'h', 'S', 'Nelson', '3/09/2017', 7.5, 3040.0, 3.0, 1.0, 2.0, 649.0, None, None, None, -37.74212, 144.89256, 'Western Metropolitan', 9264.0, 3040, 114.14285714285714, 56, 787.5, 3366.6666666666665)
('Essendon North', '89a Kerferd St', 3, 'u', 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Ejercicio 2: 

1. Seleccionar un subconjunto de columnas que les parezcan relevantes al problema de predicción del valor de la propiedad. Justificar las columnas seleccionadas y las que no lo fueron.
 - Eliminar los valores extremos que no sean relevantes para la predicción de valores de las propiedades.

2. Agregar información adicional respectiva al entorno de una propiedad a partir del [conjunto de datos de AirBnB](https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv) utilizado en el práctico. 
  1. Seleccionar qué variables agregar y qué combinaciones aplicar a cada una. Por ejemplo, pueden utilizar solo la columna `price`, o aplicar múltiples transformaciones como la mediana o el mínimo.
  1. Utilizar la variable zipcode para unir los conjuntos de datos. Sólo incluir los zipcodes que tengan una cantidad mínima de registros (a elección) como para que la información agregada sea relevante.
  2. Investigar al menos otras 2 variables que puedan servir para combinar los datos, y justificar si serían adecuadas o no. Pueden asumir que cuentan con la ayuda de anotadores expertos para encontrar equivalencias entre barrios o direcciones, o que cuentan con algoritmos para encontrar las n ubicaciones más cercanas a una propiedad a partir de sus coordenadas geográficas. **NO** es necesario que realicen la implementación.

Pueden leer otras columnas del conjunto de AirBnB además de las que están en `interesting_cols`, si les parecen relevantes.

## Ejercicio 3:

Crear y guardar un nuevo conjunto de datos con todas las transformaciones realizadas anteriormente.

## Ejercicios opcionales:

1. Armar un script en python (archivo .py) [ETL](https://towardsdatascience.com/what-to-log-from-python-etl-pipelines-9e0cfe29950e) que corra los pasos de extraccion, transformacion y carga, armando una funcion para cada etapa del proceso y luego un main que corra todos los pasos requeridos.

2. Armar un DAG en Apache Airflow que corra el ETL. (https://airflow.apache.org/docs/apache-airflow/stable/tutorial.html)